Al tener multiples procesos corriendo aparece la necesidad de sincronizar y comunicar esos procesos e hilos.

Supongamos que tenemos un recurso que no puede ser utilizado por dos personas a la vez. No puedo imprimir media pagina de una hoja y media pagina de otra por ejemplo. Que hago si tengo multiples procesos que quierenn usar ese dispositivo? tengo que serializar. Normalmente uno trata de serializar esos procesos para que se hagan de a uno a la vez. Un codigo que podriamos usar para llegar a cargo esta tarea. Tenemos una variable que se llama flag, 0 si el recurso esta libre 1 si esta usado. Levanto en el registro AX la variable flag, testeo, si no es 0 salto a ocupado, si es 0 incremento el RAX, y lo muevo a la variable flag, a partir de ahi empiezo a utilizar el recurso. En algun momento terminare de utilizarlo, borro el registro AX y grabo el 0 en flag. Si hay una sola tarea no habria problema con ese codigo.

Ahora que pasa si tenemos dos threads ejecutando el codigo anterior. Se fija si el flag esta en 0 para saber si esta libre, si no esta en 0 se altera y si esta en 0 lo pone en 1, lo usa y luego lo pone en 0 devuelta. Se ejecutan algunos pedazos del codigo de los threads que estan ejecutables. El problema es que habra un uso simultaneo de recursos, que es justamente lo que queriamos evitar. 

## Task switch

El task switch es siempre algo que se da al final de una instruccion, no a la mitad de una. Veamos el siguiente codigo. Cargo AX con 1, lo intercambio en una unica instruccion con el flag, en AX quedo el valor del flag y en flag quedo un 1, ahora yo puedo comparar ver si estaba ocupado o no. La idea es en vez de levantar el valor del flag, compararlo, hacer el jump condicional y despues ponerle un 1 al flag, lo que hace es pone un 1 en el registro y lo intercambia con el flag, ahora va a mirar el registro que en realidad tiene el contenido del flag, si el registro estaba en 0 quiere decir que yo lo pase de 0 a 1 y que por ende lo tengo yo, si el registro estaba en 1 ya estaba en 1 cuando lo intercambie por ende lo tiene otro. Porque funciona esto? la funcion exchange es una funcion unica, y por ende cuando el SO hace un task switch lo hace al final de cada instruccion. Si no se ejecuto el exchange porque llego un task swithc queda el RAX en 1 y le toca al otro porque me toma el recurso. Si ya se ejecuto el exchange cuando me interrumpen, el recurso ya es mio y cuando me devuelvan el control lo usare y cuando termine lo liberare, pero no habra un uso simultaneo del recurso. Este tipo de instrucciones se llama Read-Modify-Write, son en un solo paso.

Esto funciona siempre?

Ahora tenemos dos cores, en un core ejecuta el thread1 y en otro core el thread2. Por ejemplo el thread1 va a ejecutar el exchange, el CPU1 le dije al manejador de memoria (MMU) lee la posicion 3, este lee de memoria, levanta el valor y se lo devuelve a la CPU1, ahora simultaneamente el otro core va a ejecutar el exchange tambien, el MMU lo lee de memoria y se lo devuelve a la CPU2, el problema es que a ambos le devolvio el valor 0. Los dos RAX quedaron en 0 y otra vez tenemos el problema que se pisan entre si, el problema es que cuando tengo multiples procesadores las instrucciones se ejecutan en paralelo entonces el read-modify-write ya no funciona. Estos errores de programacion son dificiles de encontrar y es mucho mas razonable pensar 10 veces antes de hacerlo porque lleva mucho tiempo. 

## Lock

Podemos pensar en la siguiente alternativa, el prefijo LOCK. Este prefijo bloquea el bus. Cuando se va a ejecutar la ejecucion de la instruccion el bus va a quedar para mia, no pasara mas esto de que el MMU me atiende a mi simultaneamente que antiende a otros cores, de ultima si la MMU esta ocupado porque otro tiene el lock a mi me frena y la instruccion no se ejecuta. Este LOCK lo que garantiza es que el cambio es atomico, todo se produce en un unico modulo que no puede ser roto en subpedazos. Este prefijo tiene esa funcion y es lo unico que da la solucion en un ambiente multicore. Es tan comun olvidarse el LOCK que ya hoy en dia el mismo procesador agrega el LOCK como prefijo de exchange.

## Spinlocks

Que distintos mecanismos nos ofrece el SO para sincronizar las tareas sin necesidad de escribir codigo assembler o instrucciones raras. El SO nos da varios mecanismos para sincronizar tareas y tener acceso exlusivo a un recurso. El priimero es el spinlock, es como una variable que utilizara el SO y que en principio tiene como valores 0 o 1. El SO tiene que darme funciones para que pueda manejar esa variable, en windows defino una var con un spinlock. Cuando le quiero poner el 0 al principio para inicializar y decir que el recurso no esta utilizado le pongo un 0 y listo. Cuando la quiero intentar setear existe el interlockexchange que hace el lock exchange adentro, esta funcion garantiza que ningun otro thread te va a poder tocar. Cuando lo quiero liberar ejecuto el interlockexchange() en 0, el valor que devuelve lo tiro, esta es la forma de hacerlo en windows. La realidad es que es muy raro usar spinlocks. En Linux es distinto, el spinlock es una estructura, lo inicializo pasandole un puntero y poniendole un 0, tengo dos funciones, una que es el trylock y una funcion lock.

## Semaforo

Otra alternativa es el semaforo. Son parecidos pero tienen 0 (inactivo) y tienen multiples valores positivos. Es muy util porque cuando esperamos un semaforo, si esta en 0 se queda esperando, cuando algun otro thread incrementa el semaforo, arbitrariamente se elije algun lo de los otros que esten esperando y se le dice que siga y luego obviamente se baja a 0 devuelta. Cuando alguien incrementa nuevamente el semaforo, el SO agarra otra tarea arbitrariamente y le dice que siga y lo vuelve a llevar a 0. Si alguien lo incrementara a 3, el SO agarra 3 de los que esten esperando arbitrariamente y los haace seguir. La ventaja que tiene el semaforo frente al spinlock es que uno puede controlar cantidades, cada vez que se incremento el semaforo un thread que estaba esperando en ese semaforo es liberado. Como se usan? en windows hay que tener un HANDLE, el primero tiene que crear y definir la estructura del semaforo y los otros abrir la estructura del semaforo, conectarse al semaforo. Cuando uno termina hay que hacer el CloseHandle(). En linux es una estructura y la dinamica es similar. Muchas veces los lenguajes de programacion tienen sus propias bilbiotecas que exponen estas funciones con los mismos nombres. El uso mas importante de los semaforos son los thread pools.

## Mutex

Aparte de los spinlocks y los semaforos (semaforos serian un caso general de los spinlocks) tenemos otra estructura que son los Mutex. Los mutex permiten que un bloque de codigo no pueda ser ejecutado simultaenamente por dos threads. En definitiva es una estructura en memoria, hay que decirle al SO que la inicialice. Y podemos esperarlo, pero lo importante de esto es que yo vengo ejecutando, pongo el wait for single object y yo se que cuando salga de esa instruccion lo tengo yo, cuando hago release mutex al final lo libere. Hay que tener cuidado con uno programa esto, normalmente las tareas requieren datos y devuelven datos, de alguna forma deberia cargar los datos en una estructura, pasarle el puntero al thread y cuando termina devolver. De alguna forma esto involucra poner los datos en una lista encadenada.

Todos estos mecanismsos podrian combinarse.

El objetivo de los spinlocks, mutex y semaforos es coordinar los threads dentro de un mismo proceso.

## Pipes

Mecanismo de comunicacion entre proceso. En el pipe uno puede poner cosas y le van a salir del otro lado en el orden en que se pusieron. Es importante entender que cuando crea un pipe crea dos extremos, el extremo de escritura y el extremo de lectura. Cuando la creo a mi me devuelve dos descriptores que permiten escribir o leer del pipe. En windows tenes createnamedpipe que permite crear un pipe, en linux se llama mkfifo, para abrirla en windows uso createfile y en linux hago open. Se envian mensajes y se reciben mensajes. El pipe es una estructura del SO que permite conectar distintos procesos. Tecnicamente el pipe es algo de dos extremos, por ende esta pensado para conectar dos procesos. Yo podria conectar un proceso y del otro lado poner dos procesos? no es muy practico, no dara error, pero no es practico porque el mensaje se pierde cuando es leido por uno de los procesos. La realidad es que se puede poner dos procesos en una punta pero lo normal es que hayan dos procesos uno en cada punta, uno escribe otro lee y listo. Cuando un proceso arranca en linux tiene conectado tres archivos stdin: recibe datos del teclado, stdout: pantalla, y el stderr: donde se guardan los errores. Cuando uno ejecuta ls | grep "a.txt" busca econtrar el texto "a.txt". En definitiva lo que pretenedemos con esto es que se ejecute ls, que el ls genere el listado pero que se lo pase al grep para que analice linea por linea y se fije si esta "a.txt". El primer programa el ls va a tener el stdin en el teclado, va a tener el stdout en una pata de un pipe que va a crear y el stderr va a valer la pantalla. Cuando ejeceute el ls si el ls detecta un error y lo imprime el stderr va a salir en la pantalla. Pero sino, todo lo que imprima en el stdout (listado de archivos del directorio) no van a ir a la pantalla, van a entrar en una pata del pipe. Y cual van a ser los descrpitores del segundo programa (grep), el grep va a tener el stdin en la pata de salida del pipe, va a tener el stdout en la pantalla y el stderr en la pantalla tambien. Entonces el grep lo que va a procesar es lo que viene del stdin pero el stdin tiene el stdout del otro.

## Signals

Cuando uno hace un programa, independiente del punto de entrada que empiece a ejecutar el programa, el SO tiene una funcion que uno tiene que escribir si o si, y que recibe notificaciones del SO. Por ejemplo, el caso de control c control break, etc. Cuando en un proceso ponemos control-c el proceso se corta, porque se corta? el SO llama a una funcion que llama a exitprocess, podria modificar esta funcion y cuando lo notifica si es un control c y no matarlo. En el caso de wiindows las notificaciones son bastantes limitadas: control-c, control-brake, LogOff Y Shutdown. En el caso de linux son mas de 30, pero lo mas importante es que no solamente las envia el SO. Cualquier tarea con el debido privilegio puede enviar seniales a otro prooceso para notificarlo. La senial SIGUSR1 es un caso tipico que es interpretada distinto por distintos programas dado que es de propositos generales. Bajo Linux las seniales pueden ser enviadas, inclusive, dessde la consola.

## Resumen de las ultimas dos clases:

La diferencia entre los threads y los procesos es que los procesos son una entidad mas compleja entre los cuales se proteje memoria, se los considera entidades independientes dentro de la maquina. Los threads en cambio tambien representaban un hilo de ejecucion con su propio stack y su propio juego de registros, pero entre los threads del mismo proceso no habia proteccion de memoria o de archivos. Cuando el SO quiere pasar de un thread a otro la cantidad de cosas que tiene que modfiicar en el procesador para hacer ese cambio es menor, mientras que cambie el stack y los registros perfectamente termino con lo que tenia que hacer. Cuando uno conmuta de un proceso a otro todo lo que es proteccion tambien tiene que ser reinicializado para indicar que este proceso no puede acceder a los del anterior. Hace falta involucrar al hardware en este proceso de proteccion de memoria. 

